# Shunting Yard Algorithm

convert infix to postfix


In [111]:
import re

def solve_or_in_brackets(regex):
    # loop over the string to add | between any two consecutive characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            regex[i] = '('
            while(regex[i] != ']'):
                if regex[i].isalnum() and regex[i+1].isalnum():
                    regex.insert(i+1, '|')
                i += 1
            regex[i] = ')'
        else :
            i = i + 1
    return ''.join(regex)


def solve_ranges_in_brackets(regex):
    # loop over the string to add ranges of characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            regex[i] = '('
            while(regex[i] != ']'):
                if regex[i] == '-':
                    first = regex[i-1]
                    last = regex[i+1]
                    chars_between = [chr(i) for i in range(ord(first) + 1, ord(last))]
                    str = []
                    for c in chars_between:
                        str.append("|")
                        str.append(c)
                    str.append('|')
                    regex[i:i+1] = str
                    print(''.join(regex))
                i += 1
            regex[i] = ')'
        else :
            i = i + 1
    return ''.join(regex)

def replace_square(regex):
    regex = solve_or_in_brackets(regex)
    # regex = solve_ranges_in_brackets(regex)
    return regex

# def replace_square(regex):
#     i = 0
#     regex = list(regex)
#     while(i < len(regex)):
#         if regex[i] == '[':
#             regex[i] = '('
#             while(regex[i] != ']'):
#                 if regex[i] == '-':
#                     first = regex[i-1]
#                     last = regex[i+1]
#                     chars_between = [chr(i) for i in range(ord(first) + 1, ord(last)+1)]
#                     str = []
#                     for c in chars_between:
#                         str.append("|")
#                         str.append(c)
#                     str.append('|')
#                     regex[i:i+2] = str
#                 else : 
#                     # print(regex[i],regex[i+1])
#                     if regex[i] != '(' and regex[i] != '|' and regex[i+1] != '|' and regex[i+1] != '-': 
#                         regex.insert(i+1, '|')
#                 i += 1
#             regex[i-1:i+1] = ')'
#         else :
#             i = i + 1
#     return ''.join(regex)

def replace_dot(regex):
    result = []
    regex = ''.join(regex)
    
    for curr in regex:
        if curr == '.':
            result.append("[a-zA-Z0-9]")
        else : 
            result.append(curr)
    
    return ''.join(result)


def add_concat(regex):
    result = []
    prev = None

    for curr in regex:
        if prev and (prev.isalnum() or prev in ('*', '+', ')','?')) and (curr.isalnum() or curr == '(' ):
            result.append('·')  
        result.append(curr)
        prev = curr
    return ''.join(result)

def preprocessing(regex):

    regex = replace_dot(regex)
    print("\nRegex after replacing dot  : ")
    print(regex)
    regex = replace_square(regex)
    print("\nRegex after replacing square brackets  : ")
    print(regex)
    # regex = add_concat(regex)
    print("\nRegex after adding concatination  : ")
    print(regex)    
    return regex



def infix_to_postfix(infix):
    try:
        re.compile(infix)
    except re.error:
        print("Invalid regex")
        return None
    # Add implicit concatenation operator ('.')
    output = []
    for i, char in enumerate(infix):
        output.append(char)
        
        if i < len(infix) - 1:
            next_char = infix[i + 1]
            if char not in "(|.-" and next_char not in ")*+?|)-":
                output.append('.')
    infix = output

    precedence = { '*' : 5, '+' : 4, '?' : 3 , '.' : 2 , '|' : 1 , '(' : 0 }

    infix = list(infix)
    stack = []
    postfix = []
    # for i,char in enumerate(infix):
        
    #     if char == '(':
    #         stack.append(char)
        
    #     elif char == ")":
    #         stack_top = stack.pop()
    #         while stack_top != "(":
    #             postfix.append(stack_top)
    #             stack_top = stack.pop()
        
    #     elif char in ['*','+','?','.','|']:
    #         stack_top = None
    #         if len(stack) > 0:
    #             stack_top = stack.pop()
    #         while stack_top != None  and precedence[char] <= precedence[stack_top]:
    #             postfix.append(stack_top)
    #             if len(stack) != 0:
    #                 stack_top = stack.pop()
    #             else :
    #                 break
    #         if stack_top != None and precedence[char] > precedence[stack_top]:
    #             stack.append(stack_top)
    #         stack.append(char)
        
    #     else:
    #         if i < len(infix) - 1:
    #             next_char = infix[i + 1]
    #             if next_char == '-':
    #                 postfix.append(char + next_char + infix[i + 2])
    #                 i += 2
    #         else:
    #             postfix.append(char)


    i = 0
    while i < len(infix):
        char = infix[i]

        if char == '(':
            stack.append(char)

        elif char == ")":
            stack_top = stack.pop()
            while stack_top != "(":
                postfix.append(stack_top)
                stack_top = stack.pop()

        elif char in ['*', '+', '?', '.', '|']:
            stack_top = None
            if len(stack) > 0:
                stack_top = stack.pop()
            while stack_top is not None and precedence[char] <= precedence[stack_top]:
                postfix.append(stack_top)
                if len(stack) != 0:
                    stack_top = stack.pop()
                else:
                    break
            if stack_top is not None and precedence[char] > precedence[stack_top]:
                stack.append(stack_top)
            stack.append(char)

        else:
            if i < len(infix) - 2 and infix[i + 1] == '-':  # Check if a range is present
                postfix.append(char + infix[i + 1] + infix[i + 2])
                i += 2  # Skip the next two characters (handled together)

            else:
                postfix.append(char)

        i += 1  # Increment `i` for the next iteration
                        
    
    while stack:
        top = stack.pop()
        if top == '(':
            print("Unbalanced parentheses")
            return None
        postfix.append(top)
    
    return ''.join(postfix)


infix = "[A-Ga-g]?o+"
# infix="(a*)*"
preprocessed_infix = preprocessing(infix)
print("\nPOSTFIX :")
print(infix_to_postfix(preprocessed_infix))



Regex after replacing dot  : 
[A-Ga-g]?o+

Regex after replacing square brackets  : 
(A-G|a-g)?o+

Regex after adding concatination  : 
(A-G|a-g)?o+

POSTFIX :
A-Ga-g|?o+.


# Postfix to nfa

In [112]:
import json

class NFA:
    def __init__(self, states, starting_state=None, accepting_states=[]):
        self.states = states
        self.starting_state = starting_state
        self.accepting_states = accepting_states

class State1:
    def __init__(self, name, accepting=False):
        self.name = name
        self.transitions = {}  # Dictionary: {symbol: [target_states]}
        self.accepting = accepting

    def add_transition(self, symbol, target_state):
        if symbol not in self.transitions:
            self.transitions[symbol] = []
        self.transitions[symbol].append(target_state)

def new_state(state_counter, accepting=False):
    state = State1(f"S{state_counter}", accepting)
    return state

def handle_character(stack, state_counter, symbol):
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)
    start.add_transition(symbol, end)
    stack.append(NFA([start, end], start, [end]))
    return state_counter + 2

def handle_concatenation(stack, state_counter):
    if len(stack) < 2:
        raise ValueError("Invalid postfix expression for concatenation")
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    for accepting_state in nfa1.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', nfa2.starting_state)
    combined_nfa = NFA(nfa1.states + nfa2.states, nfa1.starting_state, nfa2.accepting_states)
    stack.append(combined_nfa)
    return combined_nfa,state_counter

def handle_or(stack, state_counter):
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa1.starting_state)
    start.add_transition('ε', nfa2.starting_state)

    for accepting_state in nfa1.accepting_states + nfa2.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa1.states + nfa2.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_kleene_star(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', start)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_one_or_more(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', nfa.starting_state)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_zero_or_one(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def construct_nfa(postfix_regex):
    state_counter = 1
    stack = []

    i = 0
    while i < len(postfix_regex):
        if postfix_regex[i].isalnum():
            # a-zA-Z0-9
            symbol = postfix_regex[i]
            if i < len(postfix_regex) - 10 :
                symbol = ''.join(postfix_regex[i:i+11])
            if symbol == 'a-zA-Z|0-9|' :
                state_counter = handle_character(stack, state_counter, '.')
                i += 10                            
            elif i < len(postfix_regex) - 1 and postfix_regex[i+1] == '-':
                state_counter = handle_character(stack, state_counter, postfix_regex[i] + postfix_regex[i+1] + postfix_regex[i+2])
                i += 2
            else:
                state_counter = handle_character(stack, state_counter, postfix_regex[i])
        elif postfix_regex[i] == '.':
            _,state_counter=handle_concatenation(stack, state_counter)
        elif postfix_regex[i] == '|':
            state_counter = handle_or(stack, state_counter)
        elif postfix_regex[i] == '*':
            state_counter = handle_kleene_star(stack, state_counter)
        elif postfix_regex[i] == '+':
            state_counter = handle_one_or_more(stack, state_counter)
        elif postfix_regex[i] == '?':
            state_counter = handle_zero_or_one(stack, state_counter)
        else:
            raise ValueError(f"Invalid postfix_regex[i] in postfix regex: {postfix_regex[i]}")
        i += 1

    return stack.pop()



def nfa_to_json(nfa):
    nfa_dict = {
        "startingState": nfa.starting_state.name,
    }

    for state in nfa.states:
        transitions = {}
        for symbol, targets in state.transitions.items():
            if len(targets) == 1:
                transitions[symbol] = targets[0].name  # Display as a string
            else:
                transitions[symbol] = [target.name for target in targets]  # Display as a list

        transitions["isTerminatingState"] = state.accepting

        nfa_dict[state.name] = transitions

    return json.dumps(nfa_dict, indent=6, ensure_ascii=False)

def save_nfa_to_file(nfa, filename):
    nfa_json = nfa_to_json(nfa)
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(nfa_json)


# Visualizing NFA

In [113]:
from graphviz import Digraph

def visualize_nfa(nfa):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')

    # Define the starting state
    dot.node(nfa.starting_state.name, shape='circle', style='filled', fillcolor='lightblue')

    # Define the states
    for state in nfa.states:
        shape = 'doublecircle' if state.accepting else 'circle'
        dot.node(state.name, shape=shape)

    # Add transitions
    for state in nfa.states:
        for symbol, targets in state.transitions.items():
            for target in targets:
                dot.edge(state.name, target.name, label=symbol)

    # Add an initial arrow
    dot.node('start', shape='none', label='')
    dot.edge('start', nfa.starting_state.name)

    # Save and render
    dot.render('nfa_visualization', view=False)

In [19]:
%pip install graphviz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
regex="[a-z]"
# regex = "."
preprocessed_infix = preprocessing(regex)
print("\nPOSTFIX :")
postfix_regex=infix_to_postfix(preprocessed_infix)
print(postfix_regex)

nfa = construct_nfa(postfix_regex)

visualize_nfa(nfa)

save_nfa_to_file(nfa, "nfa_output.json")




Regex after replacing dot  : 
[a-zA-Z0-9]

Regex after replacing square brackets  : 
(a-z|A-Z|0-9)

Regex after adding concatination  : 
(a-z|A-Z|0-9)

POSTFIX :
a-zA-Z|0-9|


In [117]:
import json

def expand_nfa_json(input_json_path, output_json_path):
    def expand_key(key):
        symbols = []
        if key == '.':
            # Expand wildcard to all lowercase letters (customize as needed)
            lowercase = [chr(c) for c in range(ord('a'), ord('z') + 1)]
            uppercase = [chr(c) for c in range(ord('A'), ord('Z') + 1)]
            digits= [chr(c) for c in range(ord('0'), ord('9') + 1)]
            symbols = lowercase + uppercase + digits
            
        elif '-' in key:
            # Handle ranges like 'a-z'
            parts = key.split('-')
            if len(parts) != 2 or len(parts[0]) != 1 or len(parts[1]) != 1:
                return [key]  # Invalid format
            start, end = parts
            start_ord, end_ord = ord(start), ord(end)
            if start_ord > end_ord:
                return [key]  # Invalid range
            symbols = [chr(c) for c in range(start_ord, end_ord + 1)]
        else:
            symbols = [key]  # Single symbol
        return symbols

    # Load the input NFA JSON
    with open(input_json_path, 'r') as f:
        nfa_data = json.load(f)

    # Prepare the output DFA JSON structure
    dfa_data = {
        "startingState": nfa_data["startingState"]
    }

    for state_name, state_info in nfa_data.items():
        if state_name == "startingState":
            continue  # Skip metadata entry

        expanded_transitions = {}
        is_terminating = state_info.get("isTerminatingState", False)

        for symbol, target in state_info.items():
            if symbol == "isTerminatingState":
                continue  # Skip terminating state flag

            # Expand the symbol into individual characters
            expanded_symbols = expand_key(symbol)
            for s in expanded_symbols:
                expanded_transitions[s] = target

        # Add terminating state flag and transitions
        expanded_transitions["isTerminatingState"] = is_terminating
        dfa_data[state_name] = expanded_transitions

    # Save the expanded JSON
    with open(output_json_path, 'w') as f:
        json.dump(dfa_data, f, indent=4)

    print(f"Expanded DFA JSON saved to {output_json_path}")

# Example usage:
expand_nfa_json('nfa_output.json', 'nfa_to_dfa.json')

Expanded DFA JSON saved to nfa_to_dfa.json


# Part 2

In [118]:
# from output json to input of part2
import json
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            # Convert single-state transitions from string to list
            for state, transitions in data.items():
                if isinstance(transitions, dict):  # Ignore "startingState" entry
                    for symbol, target in transitions.items():
                        if symbol != "isTerminatingState" and isinstance(target, str):
                            transitions[symbol] = [target]  # Convert string to list
            
            return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON from the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
#read json file in dict
nfa_dict=read_json_file("nfa_to_dfa.json")

In [134]:
import json
import graphviz
class State:
    def __init__(self, name, elements):
        self.name = name
        self.elements = frozenset(elements)  # Ensure it's hashable

    def __hash__(self):
        return hash(self.elements)

    def __eq__(self, other):
        return self.elements == other.elements

    def __repr__(self):
        return f"State({self.name}, {sorted(self.elements)})"

class DFA:
    def __init__(self):
        self.states = {}  # Now stores State objects as values
        self.startingState = None
        self.transitions = {}
        self.acceptingStates = set()

    def add_state(self, state, is_accepting=False):
        self.states[state.name] = state
        if is_accepting:
            self.acceptingStates.add(state)

    def add_transition(self, from_state, to_state, symbol):
        if from_state.name not in self.transitions.keys():
            self.transitions[from_state.name]={}
        if symbol not in self.transitions[from_state.name].keys():
            self.transitions[from_state.name][symbol] = set()
        self.transitions[from_state.name][symbol].add(to_state.name)
        
        
    def get_transition(self, from_state):
        if from_state in self.transitions.keys():
            return self.transitions[from_state]
        return None
    def to_json(self, filename):
        dfa_json = {
            "startingState": self.startingState.name if self.startingState else None
        }

        for state in self.states.values():
            state_data = {
                "isTerminatingState": state in self.acceptingStates
            }
            transitions = self.transitions.get(state.name, {})

            # Group symbols by their target state
            target_symbols = {}
            for symbol, to_states in transitions.items():
                if not to_states:
                    continue
                target = next(iter(to_states))  # Get the single target state
                if target not in target_symbols:
                    target_symbols[target] = []
                target_symbols[target].append(symbol)

            # Convert grouped symbols into ranges and add to JSON
            for target, symbols in target_symbols.items():
                ranges = self._get_symbol_ranges(symbols)
                # Sort ranges for consistent comparison
                sorted_ranges = sorted(ranges)

                # Check if the ranges match exactly ["0-9", "A-Z", "a-z"]
                if sorted_ranges == ["0-9", "A-Z", "a-z"]:
                    state_data["."] = target  # Use "." instead
                else:
                    for r in ranges:
                        state_data[r] = target


            dfa_json[state.name] = state_data

        with open(filename, "w") as f:
            json.dump(dfa_json, f, indent=4)

        print(f"DFA saved to {filename}")

    

    def _get_symbol_ranges(self, symbols):
        single_chars = []
        others = []
        for s in symbols:
            if len(s) == 1:
                single_chars.append(s)
            else:
                others.append(s)
        
        if not single_chars:
            return others
        
        # Sort single characters by their ASCII values
        single_chars_sorted = sorted(single_chars, key=lambda x: ord(x))
        ranges = []
        current_start = single_chars_sorted[0]
        current_end = current_start

        for c in single_chars_sorted[1:]:
            if ord(c) == ord(current_end) + 1:
                current_end = c
            else:
                if current_start == current_end:
                    ranges.append(current_start)
                else:
                    ranges.append(f"{current_start}-{current_end}")
                current_start = c
                current_end = c

        # Add the last range
        if current_start == current_end:
            ranges.append(current_start)
        else:
            ranges.append(f"{current_start}-{current_end}")

        return ranges + others

    def visualize(self, output_file="dfa_visualization"):
        dot = graphviz.Digraph(format='png')
        dot.attr(rankdir='LR')

        # Draw states
        for state in self.states.values():
            shape = "doublecircle" if state in self.acceptingStates else "circle"
            dot.node(state.name, label=state.name, shape=shape)

        # Draw starting arrow
        if self.startingState:
            dot.node("start", shape="point")
            dot.edge("start", self.startingState.name)

        # Process transitions to group symbols by target
        for from_state in self.transitions:
            transitions = self.transitions[from_state]
            target_symbols = {}

            # Group symbols by their target state
            for symbol, targets in transitions.items():
                if not targets:
                    continue
                target = next(iter(targets))  # DFA has one target per symbol
                if target not in target_symbols:
                    target_symbols[target] = []
                target_symbols[target].append(symbol)

            # Create edges with compact labels
            for target, symbols in target_symbols.items():
                # Generate ranges (e.g., "a-z") from individual symbols
                ranges = self._get_symbol_ranges(symbols)
                # Sort ranges for consistent comparison
                sorted_ranges = sorted(ranges)

                # Check if the ranges are exactly ["0-9", "A-Z", "a-z"]
                if sorted_ranges == ["0-9", "A-Z", "a-z"]:
                    label = "."
                else:
                    label = ",".join(ranges)
                            
                dot.edge(from_state, target, label=label)

        dot.render(output_file)
        print(f"DFA visualization saved to {output_file}.png")

    def visualize_min(self, output_file="dfa_visualization_min"):
        dot = graphviz.Digraph(format='png')
        dot.attr(rankdir='LR')

        # Draw states
        for state in self.states.values():
            shape = "doublecircle" if state in self.acceptingStates else "circle"
            dot.node(state.name, label=state.name, shape=shape)

        # Draw starting arrow
        if self.startingState:
            dot.node("start", shape="point")
            dot.edge("start", self.startingState.name)

        # Process transitions to group symbols by target
        for from_state in self.transitions:
            transitions = self.transitions[from_state]
            target_symbols = {}

            # Group symbols by their target state
            for symbol, targets in transitions.items():
                if not targets:
                    continue
                target = next(iter(targets))  # DFA has one target per symbol
                if target not in target_symbols:
                    target_symbols[target] = []
                target_symbols[target].append(symbol)

            # Create edges with compact labels
            for target, symbols in target_symbols.items():
                ranges = self._get_symbol_ranges(symbols)
                # Sort ranges for consistent comparison
                sorted_ranges = sorted(ranges)

                # Check if the ranges are exactly ["0-9", "A-Z", "a-z"]
                if sorted_ranges == ["0-9", "A-Z", "a-z"]:
                    label = "."
                else:
                    label = ",".join(ranges)
                dot.edge(from_state, target, label=label)

        dot.render(output_file)
        print(f"Minimized DFA visualization saved to {output_file}.png")
        
        
        
    def is_accepting_state(self, state):
        for s in self.acceptingStates:
            if s.name == state:
                return True
        return False

class NFAtoDFAConverter:
    def epsilon_closure(self, NFA, state):
        closure = {state}
        stack = [state]

        while stack:
            current = stack.pop()
            if "ε" in NFA[current]:
                for target in NFA[current]["ε"]:
                    if target not in closure:
                        closure.add(target)
                        stack.append(target)

        return closure

    def convert_to_DFA(self, NFA):
        DFA_machine = DFA()

        # Step 1: Compute the starting state (ε-closure of NFA start)
        start_closure = self.epsilon_closure(NFA, NFA["startingState"])
        start_state = State("S0", start_closure)
        DFA_machine.add_state(start_state, self.is_accepting_state(NFA, start_closure))
        DFA_machine.startingState = start_state.name

        # Step 2: Process unmarked states
        unmarked_states = [start_state]
        processed_states = set()

        while unmarked_states:
            current_state = unmarked_states.pop()
            if current_state in processed_states:
                continue
            processed_states.add(current_state)

            transitions_table = {}

            # Step 3: Iterate over all input symbols (exclude ε)
            alphabet = self.get_alphabet(NFA)

            for nfa_state in current_state.elements:
                for action in alphabet:
                    if action == 'ε':
                        continue
                    if action in NFA[nfa_state]:
                        next_states = set()
                        for next_state in NFA[nfa_state][action]:
                            next_states |= self.epsilon_closure(NFA, next_state)
                        if action not in transitions_table:
                            transitions_table[action] = next_states
                        else:
                            transitions_table[action] |= next_states

            # Step 4: Create new DFA states and transitions
            for action, next_states in transitions_table.items():
                if not next_states:
                    continue

                # Ensure consistent naming for new states
                
                next_state_name = f"S{len(DFA_machine.states)}"
                # print("//////////",next_states)
                # compact_name=""
                # for state in next_states:
                #     compact_name+=state +','
                # compact_name=compact_name[:-1]
                    
                next_state = State(next_state_name, next_states)

                # Check if state already exists
                if next_state.elements not in [s.elements for s in DFA_machine.states.values()]:
                    unmarked_states.append(next_state)
                    DFA_machine.add_state(next_state, self.is_accepting_state(NFA, next_states))
                else:
                    next_state = next(filter(lambda s: s.elements == next_state.elements, DFA_machine.states.values()))

                DFA_machine.add_transition(current_state, next_state, action)

        return DFA_machine

    def get_alphabet(self, NFA):
        alphabet = set()
        for transitions in NFA.values():
            if not isinstance(transitions, str):
                for symbol in transitions.keys():
                    if symbol != "ε" and symbol != "isTerminatingState":
                        alphabet.add(symbol)
        return alphabet

    def is_accepting_state(self, NFA, state_set):
        return any(NFA[s].get("isTerminatingState", False) for s in state_set)



In [93]:
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
# dfa.visualize("dfa")
# dfa.to_json("dfa_out.json")
# Output DFA structure
# print("Starting State:", dfa.startingState)
# print("Accepting States:", dfa.acceptingStates)
# print("Transitions:")
for from_state, transitions in dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")
        


S0 --a--> {'S1'}
S0 --b--> {'S2'}
S2 --b--> {'S2'}
S2 --a--> {'S3'}
S3 --a--> {'S3'}
S3 --b--> {'S4'}
S4 --a--> {'S3'}
S4 --b--> {'S4'}
S1 --b--> {'S2'}
S1 --a--> {'S1'}


# Convert DFA to minimized DFA

In [128]:
class convertDFAToMinDFA:
    def convert_to_min_dfa(self, dfa: DFA):
        # Step 1: Separate accepting and non-accepting states
        accepting, non_accepting = self.separate_acceptance(dfa)
        
        # Step 2: Create initial partition (handle empty sets)
        partitions = []
        if accepting:
            partitions.append(accepting)
        if non_accepting:
            partitions.append(non_accepting)
        
        changed = True
        while changed:
            changed = False
            new_partitions = []
            for group in partitions:
                if len(group) <= 1:
                    new_partitions.append(group)
                    continue
                
                # Split group based on transition behavior
                transition_groups = {}
                for state in group:
                    key = self.get_transition_key(dfa, state, partitions)
                    if key not in transition_groups:
                        transition_groups[key] = set()
                    transition_groups[key].add(state)
                
                # Update partitions if split occurred
                if len(transition_groups) > 1:
                    changed = True
                    new_partitions.extend(transition_groups.values())
                else:
                    new_partitions.append(group)
            
            partitions = new_partitions
        
        # Step 3: Build minimized DFA
        return self.create_min_DFA(dfa, partitions)
    
    def get_transition_key(self, dfa: DFA, state: str, partitions: list[set]) -> frozenset:
        transitions = dfa.get_transition(state) or {}
        key = {}
        for symbol, targets in transitions.items():
            if not targets:
                continue  # Handle incomplete DFAs if needed
            target_state = next(iter(targets))
            # Find which partition contains the target state
            for partition in partitions:
                if target_state in partition:
                    key[symbol] = frozenset(partition)
                    break
        return frozenset(key.items())
    
    def separate_acceptance(self, dfa: DFA) -> tuple[set, set]:
        accepting = set()
        non_accepting = set()
        for state in dfa.states:
            if dfa.is_accepting_state(state):
                accepting.add(state)
            else:
                non_accepting.add(state)
        return accepting, non_accepting
    
    def create_min_DFA(self, dfa: DFA, partitions: list[set]) -> DFA:
        min_dfa = DFA()
        state_map = {}  # Original state name → new State
        
        # Create new states
        for i, partition in enumerate(partitions):
            state_name = f"S{i}"
            new_state = State(state_name, partition)
            min_dfa.add_state(new_state)
            
            # Map original states to new state
            for original_state in partition:
                state_map[original_state] = new_state
            
            # Set initial state
            if dfa.startingState in partition:
                min_dfa.startingState = new_state
            
            # Set accepting state
            if any(dfa.is_accepting_state(s) for s in partition):
                min_dfa.acceptingStates.add(new_state)
        
        # Add transitions using representative states
        for new_state in min_dfa.states.values():
            # Pick any state from the partition
            sample_original = next(iter(new_state.elements))
            original_transitions = dfa.get_transition(sample_original) or {}
            for symbol, targets in original_transitions.items():
                if not targets:
                    continue
                target_original = next(iter(targets))
                target_new = state_map[target_original]
                min_dfa.add_transition(new_state, target_new, symbol)
        
        return min_dfa

In [90]:
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
# print(dfa.transitions)
minDFA=convertDFAToMinDFA()
final_dfa=minDFA.convert_to_min_dfa(dfa)
final_dfa.visualize_min("dfa_min")
final_dfa.to_json("dfa_out.json")
print(final_dfa.states)
# final_dfa.to_json("min_json.json")
# Output DFA structure
# print("Starting State:", dfa.startingState)
# print("Accepting States:", dfa.acceptingStates)
# print("Transitions:")
for from_state, transitions in final_dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")
        


Minimized DFA visualization saved to dfa_min.png
DFA saved to dfa_out.json
{'S0': State(S0, ['S4']), 'S1': State(S1, ['S0', 'S2']), 'S2': State(S2, ['S3']), 'S3': State(S3, ['S1'])}
S0 --a--> {'S0'}
S1 --a--> {'S1'}
S1 --b--> {'S3'}
S2 --a--> {'S0'}
S3 --b--> {'S2'}
S3 --a--> {'S0'}


In [135]:
# final cell 
regex="[0-9a-zA-Z]"
preprocessed_infix = preprocessing(regex)
print("\nPOSTFIX :")
postfix_regex=infix_to_postfix(preprocessed_infix)
print(postfix_regex)

nfa = construct_nfa(postfix_regex)

visualize_nfa(nfa)
file_path="nfa_output.json"
save_nfa_to_file(nfa, file_path)
expand_nfa_json(file_path, "nfa_to_dfa.json")
# from NFA to DFA
nfa_dict=read_json_file("nfa_to_dfa.json")
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
# dfa.visualize("dfa")

#from DFA to minDFA
minDFA=convertDFAToMinDFA()
final_dfa=minDFA.convert_to_min_dfa(dfa)
final_dfa.visualize_min("dfa_min")
final_dfa.to_json("dfa_min.json")


Regex after replacing dot  : 
[0-9a-zA-Z]

Regex after replacing square brackets  : 
(0-9|a-z|A-Z)

Regex after adding concatination  : 
(0-9|a-z|A-Z)

POSTFIX :
0-9a-z|A-Z|
Expanded DFA JSON saved to nfa_to_dfa.json
Minimized DFA visualization saved to dfa_min.png
DFA saved to dfa_min.json
